In [7]:
import h3
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping
from concurrent.futures import ProcessPoolExecutor

In [8]:
class H3ShapefileGenerator:
    def __init__(self, shapefile_path, output_path, resolution=9):
        self.shapefile_path = shapefile_path
        self.output_path = output_path
        self.resolution = resolution
        self.polygon = self.load_shapefile().geometry[0]

    def load_shapefile(self):
        return gpd.read_file(self.shapefile_path).to_crs("EPSG:4326")

    def convert_polygon_to_h3_indices(self):
        polygon_geojson = mapping(self.polygon)
        return h3.polyfill(polygon_geojson, self.resolution)

    @staticmethod
    def convert_h3_index_to_polygon(h3_index):
        try:
            coords = h3.h3_to_geo_boundary(h3_index)
            return Polygon(coord[::-1] for coord in coords)
        except Exception as e:
            print(f"Error converting H3 index to polygon: {e}")
            return None

    def generate_shapefile_from_h3_indices(self):
        hex_df = pd.DataFrame(self.convert_polygon_to_h3_indices(), columns=['hex9'])
        with ProcessPoolExecutor() as executor:
            hex_df['geometry'] = list(executor.map(self.convert_h3_index_to_polygon, hex_df['hex9']))
        hex_df.dropna(subset=['geometry'], inplace=True)
        gdf = gpd.GeoDataFrame(hex_df, geometry='geometry', crs=4326)
        gdf.to_file(self.output_path)
        print(f"Shapefile saved at {self.output_path}")

In [9]:
if __name__ == "__main__":
    generator = H3ShapefileGenerator(
        shapefile_path="../../shapefiles/zh_poly.shp",
        output_path='../../hex/h3_list.shp'
    )
    generator.generate_shapefile_from_h3_indices()

Shapefile saved at ../../hex/h3_list.shp
